In [1]:
import numpy as np
import tensorflow as tf
import math, os, sys, datetime
from pandas_datareader import data as pdr
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
from functools import reduce
from sklearn import preprocessing
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from models.psooptimizer import Optimizer as PSOOptimizer
from utils.preprocessstock import preprocess
import tensorflow_probability as tfp
from ta import add_all_ta_features
from ta.utils import dropna

#Tickers list
# tickers=['AAPL']
tickers=['AAPL','AMZN','FB','GOOG','MMM','MSFT','NFLX','NKE','NVDA','INTC','CSCO','WMT','TSLA','EBAY',
         'ORCL','CBG.BK','OSP.BK','BBL.BK','SCB.BK','LH.BK','MINT.BK','PTT.BK','BANPU.BK','ADVANC.BK',
         'TRUE.BK','AOT.BK','BEM.BK','BTS.BK','CPALL.BK','CPN.BK']
# tickers2=['CBG.BK','OSP.BK','BBL.BK','SCB.BK','MINT.BK','PTT.BK','ADVANC.BK',
#          'AOT.BK','CPALL.BK','CPN.BK']
startdate = datetime.datetime(2017, 1, 13)
enddate = datetime.datetime(2021, 1, 1)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [10]:
def build_model(loss):
    layers = [
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(7, 3, padding='same', activation='relu', input_shape=(1, 7, 5, 1))),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(7, 3, padding='same', activation='relu', dilation_rate=2)),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(14, 3, padding='same', activation='relu', dilation_rate=1)),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(36, 3, padding='same', activation='relu', dilation_rate=1)),
        tf.keras.layers.Dense(70, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(7, activation='linear')
    ]
    model = tf.keras.models.Sequential(layers=layers)
    model.compile(optimizer='adam', loss=loss)
    return model

In [5]:
timesteps = 7
N = 10
STEPS = 1000
LOSS = 'mse'
BATCH_SIZE = 32

In [16]:
def run_backpropagation(x_train, y_train, epochs=0):
    best_model = None
    best_score = float('inf')
    if epochs == 0:
        epochs = STEPS
    for i in range(N):
        model_s = build_model(LOSS)
        model_s.fit(
            x_train, y_train,
            epochs=epochs,
            batch_size=BATCH_SIZE,
            verbose=0
        )
        train_score = model_s.evaluate(x_train, y_train, batch_size=BATCH_SIZE, verbose=0)
        if train_score < best_score:
            best_model = model_s
            best_score = train_score
        return best_model

In [17]:
model_s = build_model(LOSS)
for stocks in tickers:
    # Load Data
    ticker = yf.Ticker(stocks)
    data = ticker.history(start=startdate, end=enddate) 
    data['next_Close'] = data['Close'].shift(-7)
    data = data.drop(columns=['Dividends', 'Stock Splits'])
    data = dropna(data)
    
    # Add Indicator
    data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    data = data.drop(columns=['Volume', 'Open', 'High', 'Low'])
    
    # Feature Selection
    y = data['next_Close']
    featureScores = pd.DataFrame(data[data.columns[1:]].corr()['next_Close'][:])
    x_list = []
    for i in range(0, len(featureScores)):
        if abs(featureScores.next_Close[i]) > 0.90:
            x_list.append(featureScores.index[i])
    X = data[x_list]
    X = X.drop(columns=['next_Close'])
    sfs1 = SFS(LinearRegression(), k_features=(1,5), forward=True, floating=False, cv=0)
    sfs1.fit(X, y)
    k_feature_names = list(sfs1.k_feature_names_)
    features = data[k_feature_names]
    
    # Preporcess
    min_max_scaler = preprocessing.MinMaxScaler()
    features = min_max_scaler.fit_transform(features)
    features = features[:len(features)//timesteps*timesteps].reshape((len(features)//timesteps, timesteps, 5))
    
    labels = data[['next_Close']]
    labels = min_max_scaler.fit_transform(labels)
    labels = labels[:len(labels)//timesteps*timesteps].reshape((len(labels)//timesteps, timesteps, 1))
    labels = np.squeeze(labels)
    
    train_test_split_factor = .80
    validation_split_factor = .20
    train_x, train_y, test_x, test_y = features[:math.floor(len(features)*train_test_split_factor)], labels[:math.floor(len(labels)*train_test_split_factor)], features[math.floor(len(features)*train_test_split_factor):], labels[math.floor(len(labels)*train_test_split_factor):]
    train_x, test_x = np.expand_dims(train_x, axis=1), np.expand_dims(test_x, axis=1)
    train_x, test_x = np.expand_dims(train_x, axis=-1), np.expand_dims(test_x, axis=-1)
    input_shape = train_x.shape
    model_s.fit(
            train_x, train_y,
            epochs=STEPS,
            batch_size=BATCH_SIZE,
            verbose=0
        )
    b_train_score = model_s.evaluate(train_x, train_y, batch_size=BATCH_SIZE, verbose=0)
    b_test_score = model_s.evaluate(test_x, test_y, batch_size=BATCH_SIZE, verbose=0)
    print(f'{"="*52}')
    print(f"Stock : {stocks}")
    print("Backprop -- train: {:.4f}  test: {:.4f}".format(b_train_score, b_test_score))


Stock : AAPL
Backprop -- train: 0.0002  test: 0.0071
Stock : AMZN
Backprop -- train: 0.0009  test: 0.0191
Stock : FB
Backprop -- train: 0.0009  test: 0.0075
Stock : GOOG
Backprop -- train: 0.0007  test: 0.0085
Stock : MMM
Backprop -- train: 0.0013  test: 0.0031
Stock : MSFT
Backprop -- train: 0.0002  test: 0.0024
Stock : NFLX
Backprop -- train: 0.0010  test: 0.0056
Stock : NKE
Backprop -- train: 0.0003  test: 0.0066
Stock : NVDA
Backprop -- train: 0.0003  test: 0.0064
Stock : INTC
Backprop -- train: 0.0017  test: 0.0088
Stock : CSCO
Backprop -- train: 0.0013  test: 0.0046
Stock : WMT
Backprop -- train: 0.0004  test: 0.0025
Stock : TSLA
Backprop -- train: 0.0000  test: 0.0043
Stock : EBAY
Backprop -- train: 0.0007  test: 0.0072
Stock : ORCL
Backprop -- train: 0.0013  test: 0.0056
Stock : CBG.BK
Backprop -- train: 0.0007  test: 0.0181
Stock : OSP.BK
Backprop -- train: 0.0012  test: 0.0080
Stock : BBL.BK
Backprop -- train: 0.0009  test: 0.0058
Stock : SCB.BK
Backprop -- train: 0.0012  tes

In [9]:
model_s.save('weights/tdnn.h5')
model_s.save_weights('weights/tdnn_weight_only.h5')

In [10]:
model_p = build_model(LOSS)
for stocks in tickers:
    # Load Data
    ticker = yf.Ticker(stocks)
    data = ticker.history(start=startdate, end=enddate)
    # data.insert(0, 'ticker', stocks)    
    data['next_Close'] = data['Close'].shift(-7)
    data = data.drop(columns=['Dividends', 'Stock Splits'])
    data = dropna(data)
    # Add Indicator
    data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    data = data.drop(columns=['Volume', 'Open', 'High', 'Low'])
    # Feature Selection
    y = data['next_Close']
    featureScores = pd.DataFrame(data[data.columns[1:]].corr()['next_Close'][:])
    x_list = []
    for i in range(0, len(featureScores)):
        if abs(featureScores.next_Close[i]) > 0.90:
            x_list.append(featureScores.index[i])
    X = data[x_list]
    X = X.drop(columns=['next_Close'])
    sfs1 = SFS(LinearRegression(), k_features=(1,5), forward=True, floating=False, cv=0)
    sfs1.fit(X, y)
    k_feature_names = list(sfs1.k_feature_names_)
    features = data[k_feature_names]
    # Perporcess
    min_max_scaler = preprocessing.MinMaxScaler()
    features = min_max_scaler.fit_transform(features)
    features = features[:len(features)//timesteps*timesteps].reshape((len(features)//timesteps, timesteps, 5))
    
    labels = data[['next_Close']]
    labels = min_max_scaler.fit_transform(labels)
    labels = labels[:len(labels)//timesteps*timesteps].reshape((len(labels)//timesteps, timesteps, 1))
    labels = np.squeeze(labels)
    
    train_test_split_factor = .80
    validation_split_factor = .20
    train_x, train_y, test_x, test_y = features[:math.floor(len(features)*train_test_split_factor)], labels[:math.floor(len(labels)*train_test_split_factor)], features[math.floor(len(features)*train_test_split_factor):], labels[math.floor(len(labels)*train_test_split_factor):]
    train_x, test_x = np.expand_dims(train_x, axis=1), np.expand_dims(test_x, axis=1)
    train_x, test_x = np.expand_dims(train_x, axis=-1), np.expand_dims(test_x, axis=-1)
    input_shape = train_x.shape
    model_p.fit(
            train_x, train_y,
            epochs=STEPS,
            batch_size=BATCH_SIZE,
            verbose=0
        )
    pso = PSOOptimizer(model=model_p,
                  loss=LOSS,
                  n=N,
                  inertia_weight=0.7298,
                  local_rate=1.49618,
                  global_rate=1.49618)
    print(f'{"="*52}')
    pso.fit(train_x, train_y, steps=STEPS, batch_size=BATCH_SIZE)
    model_p = pso.get_best_model()
    p_train_score = model_p.evaluate(train_x, train_y, batch_size=BATCH_SIZE, verbose=0)
    p_test_score = model_p.evaluate(test_x, test_y, batch_size=BATCH_SIZE, verbose=0)
    print(f"Stock : {stocks}")
    print("PSO -- train: {:.4f}  test: {:.4f}".format(p_train_score, p_test_score))

PSO -- Initial best score 0.0001
 [====================] 
Stock : AAPL
PSO -- train: 0.0001  test: 0.0115
PSO -- Initial best score 0.0002
 [====================] 
Stock : AMZN
PSO -- train: 0.0002  test: 0.0174
PSO -- Initial best score 0.0006
 [====================] 
Stock : FB
PSO -- train: 0.0006  test: 0.0191
PSO -- Initial best score 0.0005
 [====================] 
Stock : GOOG
PSO -- train: 0.0005  test: 0.0354
PSO -- Initial best score 0.0010
 [====================] 
Stock : MMM
PSO -- train: 0.0010  test: 0.0042
PSO -- Initial best score 0.0001
 [====================] 
Stock : MSFT
PSO -- train: 0.0001  test: 0.0042
PSO -- Initial best score 0.0004
 [====================] 
Stock : NFLX
PSO -- train: 0.0004  test: 0.0513
PSO -- Initial best score 0.0001
 [====================] 
Stock : NKE
PSO -- train: 0.0001  test: 0.0317
PSO -- Initial best score 0.0001
 [====================] 
Stock : NVDA
PSO -- train: 0.0001  test: 0.0200
PSO -- Initial best score 0.0007
 [===============

In [11]:
model_p.save('weights/tdnnpso.h5')
model_p.save_weights('weights/tdnnpso_weight_only.h5')